In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import glob
from pathlib import Path


object_name = "Plate1"
input_folder = "./datasets/detect/train/images"
output_folder = "./datasets/classify/train"

def resize_to_square(image, size=640):
    h, w = image.shape[:2]
    scale = size / max(h, w)
    new_w, new_h = int(w * scale), int(h * scale)
    resized = cv2.resize(image, (new_w, new_h), interpolation=cv2.INTER_LINEAR)

    square_image = np.zeros((size, size, 3), dtype=np.uint8)

    x_offset = (size - new_w) // 2
    y_offset = (size - new_h) // 2
    square_image[y_offset:y_offset + new_h, x_offset:x_offset + new_w] = resized

    return square_image

image_files = glob.glob(f"{input_folder}/image_{object_name}_*.jpg")

for file in image_files:
    image = cv2.imread(file)
    image[:200,:] = image[0,0]
    image[:,1750:] = image[0,0]

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, binary = cv2.threshold(gray, 100, 255, cv2.THRESH_BINARY_INV)

    kernel = np.ones((5, 5), np.uint8)
    binary = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)

    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    min_area = 5000
    max_area = 200000
    filtered_contours = [cnt for cnt in contours if min_area < cv2.contourArea(cnt) < max_area]

    for contour in filtered_contours:
        x, y, w, h = cv2.boundingRect(contour)
        image_filtered = image[y:y + h + 1, x:x + w + 1]

    image_filtered = resize_to_square(image_filtered)

    plt.figure(figsize=(8, 6))
    plt.imshow(cv2.cvtColor(image_filtered, cv2.COLOR_BGR2RGB))
    plt.show()

    cv2.imwrite(f"{output_folder}/{object_name}/{Path(file).name}", resize_to_square(image_filtered))